In [1]:
import pyspark
from pyspark import SparkConf, SparkContext


from pyspark.sql import SparkSession, SQLContext, Row


In [2]:
import numpy as np
from itertools import islice

In [3]:
from pyspark.ml.linalg import Vector, Vectors, VectorUDT,SparseVector

from pyspark.mllib.feature import Word2Vec, IDF, HashingTF
from pyspark.mllib.linalg import Vector, Vectors, VectorUDT,SparseVector

In [1]:
import findspark
findspark.init()

In [2]:
# basically spark
import pyspark
import operator
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.window import Window
from pyspark.sql.types import ArrayType, FloatType, StringType, IntegerType
from pyspark.sql.functions import udf, row_number,column

# processing
import re
from datetime import datetime

# text preprocessing
import nltk
from nltk.corpus import stopwords

#staff for LDA
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vector as oldVector, Vectors as oldVectors
from pyspark.ml.linalg import Vector as newVector, Vectors as newVectors

#for debug purpose only
import time

In [3]:
# text preprocessing
from pyspark.ml.feature import CountVectorizer,StopWordsRemover, HashingTF, IDF, Tokenizer

#staff for LDA
from pyspark.mllib.linalg import Vector as oldVector, Vectors as oldVectors
from pyspark.ml.linalg import Vector as newVector, Vectors as newVectors

#for debug purpose only
import time

In [4]:
num_rows_to_show = 20
text_file = 'data/listings.csv'

In [5]:
sc = SparkContext()
spark = SparkSession(sc)

In [8]:
df = spark.read.csv(text_file, inferSchema=True, header=True)
df = df.select("id", "name").dropna(subset="name")

In [9]:
print(time.strftime('%m%d%Y %H:%M:%S'))

tokenizer = Tokenizer(inputCol="name", outputCol="words")
df = tokenizer.transform(df)
print(time.strftime('%m%d%Y %H:%M:%S'))

07042019 22:47:48
07042019 22:47:48


In [10]:
print(time.strftime('%m%d%Y %H:%M:%S'))

cv = CountVectorizer(inputCol="words", outputCol="raw_features", vocabSize=5000, minDF=2.0)
cvmodel = cv.fit(df)

print(time.strftime('%m%d%Y %H:%M:%S'))

07042019 22:47:49
07042019 22:47:50


In [11]:
print(time.strftime('%m%d%Y %H:%M:%S'))
df = cvmodel.transform(df)
print(time.strftime('%m%d%Y %H:%M:%S'))

07042019 22:47:50
07042019 22:47:50


In [12]:
df.show(10, False)

+-----+-------------------------------------------------+---------------------------------------------------------+--------------------------------------------------------------------+
|id   |name                                             |words                                                    |raw_features                                                        |
+-----+-------------------------------------------------+---------------------------------------------------------+--------------------------------------------------------------------+
|2818 |Quiet Garden View Room & Super Fast WiFi         |[quiet, garden, view, room, &, super, fast, wifi]        |(2674,[8,9,17,31,51,141,237,1189],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|20168|100%Centre-Studio 1 Private Floor/Bathroom       |[100%centre-studio, 1, private, floor/bathroom]          |(2674,[20,103,2583,2615],[1.0,1.0,1.0,1.0])                         |
|25428|Lovely apt in City Centre (Jordaan)              |[lovely, apt, in, 

In [13]:
df = df.drop("name")

In [15]:
w = Window().orderBy(column("id"))
df = df.withColumn("id", row_number().over(w))

In [16]:
rs = df.rdd.map(lambda x: (x[0], x[1], oldVectors.fromML(x[2])))

In [17]:
rs.take(10)

[(1, ['27987182'], SparseVector(2674, {})),
 (2, ['21686664'], SparseVector(2674, {2292: 1.0})),
 (3, ['21686664'], SparseVector(2674, {2292: 1.0})),
 (4, ['17607060'], SparseVector(2674, {})),
 (5, ['5630387'], SparseVector(2674, {})),
 (6,
  ['yays', 'bickersgracht', '2-bedroom', 'c'],
  SparseVector(2674, {265: 1.0, 693: 1.0, 770: 1.0, 952: 1.0})),
 (7,
  ['yays', 'bickersgracht', '2-bedroom', 'e'],
  SparseVector(2674, {265: 1.0, 693: 1.0, 952: 1.0, 1080: 1.0})),
 (8,
  ['roof', 'terrace', 'cozy', 'place'],
  SparseVector(2674, {16: 1.0, 45: 1.0, 62: 1.0, 84: 1.0})),
 (9,
  ['amazing', 'apt', 'next', 'to', 'rembrandtpark', '&close', 'to', 'center'],
  SparseVector(2674, {13: 2.0, 14: 1.0, 32: 1.0, 58: 1.0, 72: 1.0, 334: 1.0})),
 (10,
  ['yays', 'oostenburgergracht', 'studio', 'a'],
  SparseVector(2674, {26: 1.0, 28: 1.0, 693: 1.0, 722: 1.0}))]

In [19]:
rs.toDF().show(10)

+---+--------------------+--------------------+
| _1|                  _2|                  _3|
+---+--------------------+--------------------+
|  1|          [27987182]|        (2674,[],[])|
|  2|          [21686664]| (2674,[2292],[1.0])|
|  3|          [21686664]| (2674,[2292],[1.0])|
|  4|          [17607060]|        (2674,[],[])|
|  5|           [5630387]|        (2674,[],[])|
|  6|[yays, bickersgra...|(2674,[265,693,77...|
|  7|[yays, bickersgra...|(2674,[265,693,95...|
|  8|[roof, terrace, c...|(2674,[16,45,62,8...|
|  9|[amazing, apt, ne...|(2674,[13,14,32,5...|
| 10|[yays, oostenburg...|(2674,[26,28,693,...|
+---+--------------------+--------------------+
only showing top 10 rows



In [33]:
# Run the LDA Topic Modeler
# Note the time before and after is printed in order to find out how much time it takes to process x number of records

print(time.strftime('%m%d%Y %H:%M:%S'))
num_topics = 5
max_iterations = 20

lda_model = LDA.train(rs_df['index', 'raw_features'].rdd.map(list), k=3, maxIterations=max_iterations)
print(time.strftime('%m%d%Y %H:%M:%S'))

07042019 22:18:18
07042019 22:18:33


In [63]:
topicIndices.toDF().show(10,True)

+--------------------+--------------------+
|                  _1|                  _2|
+--------------------+--------------------+
|[0, 1, 2, 3, 4, 6...|[0.06455992319820...|
|[0, 1, 2, 3, 4, 5...|[0.06395919915980...|
|[0, 1, 2, 3, 4, 5...|[0.06382422394248...|
+--------------------+--------------------+



### Show topics

In [20]:
topics = lda_model.topicsMatrix()
vocabArray = cvmodel.vocabulary

wordNumbers = 20  # number of words per topic
topicIndices = sc.parallelize(lda_model.describeTopics(maxTermsPerTopic = wordNumbers))

def topic_render(topic):  # specify vector id of words to actual words
    terms = topic[0]
    prob = topic[1]
    
    result = []
    for i in range(wordNumbers):
        term = vocabArray[terms[i]]+":  "+str(round(prob[i],3))
        result.append(term)
    return result

NameError: name 'lda_model' is not defined

In [ ]:
print(time.strftime('%m%d%Y %H:%M:%S'))
topics_final = topicIndices.map(lambda topic:topic_render(topic)).collect()
print(time.strftime('%m%d%Y %H:%M:%S'))

In [36]:
for topic in range(len(topics_final)):
    print ("Topic #" + str(topic+1) + "")
    for term in topics_final[topic]:
        print (term)
    print ('\n')